# Libraries

In [1]:
import jupyternotify
import os
import pandas as pd
import pandas.io.formats.excel
from datetime import datetime
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pandas.io.formats.excel.ExcelFormatter.header_style = None

ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

# Importación

Al mes el cliente nos manda 2 archivos en vez de 1:

- Inventario y Ventas por punto de venta (df_pdv)
- Inventario por CEDIS y ventas por sucursal (df_suc)

In [2]:
%%time
anio = 2021
semana = 9
#path = '../../../../../../../../genommalabinternacional/Cristian Javier Sanchez Yepez - NoB2B/Copservir/Copservir Faltante/2021/S 9/'
#path = '../../../../1Data/2Catalogue/'
path = r"C:\Users\jshernandezm\genommalabinternacional\Cristian Javier Sanchez Yepez - NoB2B\Copservir\Copservir Faltante\2021\S {0}"
df_pdv = pd.read_csv(path.format(semana) + '\\Reporte_Inventario_Ventas_PDV.csv', encoding='latin1')
df_suc = pd.read_excel(path.format(semana) + '\\Reporte_Inventario_Ventas_Sucursal.xlsx', sheet_name='INVENTARIOS SUCURSALES', header=4)

Wall time: 787 ms


# Transformación inventarios por sucursal

In [3]:
data_suc = df_suc[['Unnamed: 0', 'Unnamed: 1', 'BQUILLA', 'BOGOTA', 'BMANGA', 'CALI']].copy()

data_suc.rename({'BQUILLA':'BAR', 'BOGOTA':'BOG', 'BMANGA':'BUC', 'CALI':'CAL'}, axis=1, inplace=True)
data_suc.rename(dict(zip(data_suc.columns[:2], data_suc.loc[1].tolist()[:2])), axis=1, inplace=True)

data_suc.drop([0, 1], axis=0, inplace=True)
data_suc.reset_index(drop=True, inplace=True)
# Datos por sucursal transformados
data_suc_t = data_suc.melt(id_vars=['Codigo', 'Descripcion'], value_vars=['BAR', 'BOG', 'BUC', 'CAL'], var_name='codHoml', value_name='Cantidad Saldo')

In [4]:
data_suc[:5]

,Codigo,Descripcion,BAR,BOG,BUC,CAL
0,15512,JABON ASEPXIA NEUTRO BARRA X 100 GRS,0,0,0,0
1,15706,UNESIA CREMA ANTIMICOTICO TUBO X 20 GR,327,399,271,371
2,27543,MEDICASP CHAMPU CAPILAR ANTICASPA KETOCONAZOL ...,559,513,409,568
3,63944,CREMA SECANTE ASEPXIA ANTI IMPERFECCIONES CAMO...,0,0,0,0
4,63946,JABON ASEPXIA FORTE BARRA X 100 GRS,0,0,0,0


In [5]:
data_suc_t['Mostrador'] = data_suc_t['codHoml'].map({'BAR':'BARRANQUILLA', 'BOG':'BOGOTA', 'BUC':'BUCARAMANGA', 'CAL':'CALI'})
data_suc_t['so'] = 0

In [6]:
data_suc_t

,Codigo,Descripcion,codHoml,Cantidad Saldo,Mostrador,so
0,15512,JABON ASEPXIA NEUTRO BARRA X 100 GRS,BAR,0,BARRANQUILLA,0
1,15706,UNESIA CREMA ANTIMICOTICO TUBO X 20 GR,BAR,327,BARRANQUILLA,0
2,27543,MEDICASP CHAMPU CAPILAR ANTICASPA KETOCONAZOL ...,BAR,559,BARRANQUILLA,0
3,63944,CREMA SECANTE ASEPXIA ANTI IMPERFECCIONES CAMO...,BAR,0,BARRANQUILLA,0
4,63946,JABON ASEPXIA FORTE BARRA X 100 GRS,BAR,0,BARRANQUILLA,0
...,...,...,...,...,...,...
875,13264,POLVO ASEPXIA MATIZANTE,CAL,0,CALI,0
876,13262,CREMA CORPORAL CICATRICURE ESPECIALIZADA REAFI...,CAL,0,CALI,0
877,13261,CREMA CORPORAL CICATRICURE ESPECIALIZADA PREVE...,CAL,0,CALI,0
878,13258,CREMA CORPORAL CICATRICURE ESPECIALIZADA PREVE...,CAL,0,CALI,0


In [7]:
data_suc_t['Cantidad Saldo'].sum()

63974

# Transformación inventarios y ventas por PDV

In [8]:
data_pdv = df_pdv[['PDV', 'NOMBRE PDV', 'COD', 'DESCRIPCION', 'Inv Und Total', 'Vta Und Total']].copy()

# Concatenación de inventario y stock

In [9]:
def homolog_codigos(codigo):
    
    dict_locales = {'400000000':'4E8', '4000000':'4E6', '400000':'4E5',
                    '40000':'4E4', '4000':'4E3', '400':'4E2', '40':'4E1',
                    '3000000000':'3E9', '300000000':'3E8', '30000000':'3E7',
                    '3000000':'3E6', '300000':'3E5', '30000':'3E4', 
                    '3000':'3E3', '30':'3E1', '200000000':'2E8', '20000000':'2E7',
                    '200000':'2E5', '20000':'2E4', '200':'2E2',  '20':'2E1',
                    '1000000000':'1E9', '100000000':'1E8', '10000000':'1E7',
                    '100000':'1E5', '10000':'1E4', '1000':'1E3', '100':'1E2', '10':'1E1'}
    
    try:
        return dict_locales[codigo]
    except:
        return codigo

In [10]:
data_pdv['PDV'] = data_pdv['PDV'].astype(str)
data_pdv['PDV'] = data_pdv['PDV'].map(homolog_codigos)

In [11]:
data_suc_t.rename({'Codigo':'Código producto', 'Descripcion':'Producto'}, axis=1, inplace=True)

In [12]:
data_pdv.rename({'PDV':'codHoml', 'NOMBRE PDV':'Mostrador', 'COD':'Código producto', 'DESCRIPCION':'Producto', 'Inv Und Total':'Cantidad Saldo', 'Vta Und Total':'so'}, axis=1, inplace=True)
# Concatenación de ambos datos (pdv y suc)
data = pd.concat([data_pdv, data_suc_t])
data['Cantidad Saldo'] = data['Cantidad Saldo'].astype(float)

In [13]:
data['Cantidad Saldo'].sum()

229061.79

In [42]:
data['so'].sum()

45416.92

In [14]:
data.isnull().sum()

codHoml            0
Mostrador          0
Código producto    0
Producto           0
Cantidad Saldo     0
so                 0
dtype: int64

In [15]:
data = data.pivot_table(index=['codHoml', 'Mostrador', 'Código producto', 'Producto'], values=['Cantidad Saldo', 'so'], aggfunc='sum').reset_index()

# Desacumulación

In [16]:
%%time
path = '../../../../../../../02Paises/Colombia/Copservir/Layout/'
#files = ['Copservir_2021_10.xlsx', 'Copservir_2021_11.xlsx', 'Copservir_2021_12.xlsx', 'Copservir_2021_13.xlsx']
files = ['Copservir_2021_06.xlsx', 'Copservir_2021_07.xlsx', 'Copservir_2021_08.xlsx']

df_acum_ant = pd.DataFrame()
for f in files:
    df_acum_ant = pd.concat([df_acum_ant, pd.read_excel(path + f)])

Wall time: 15.5 s


In [17]:
df_acum_ant.isnull().sum()

codHoml            0
Mostrador          0
Código producto    0
Producto           0
Cantidad Saldo     0
so                 0
dtype: int64

In [18]:
data_acum_ant = df_acum_ant.pivot_table(index=['codHoml', 'Mostrador', 'Código producto', 'Producto'], values=['Cantidad Saldo', 'so'], aggfunc='sum').reset_index()

In [38]:
df_acum_ant.shape

(240540, 6)

In [19]:
data_acum_ant.shape

(85849, 6)

In [20]:
data_acum_ant['codHoml'] = data_acum_ant['codHoml'].astype(str)
data_acum_ant['Código producto'] = data_acum_ant['Código producto'].astype(str)
data_acum_ant['ID'] = data_acum_ant['codHoml'] + data_acum_ant['Código producto']

Nos quedamos con un solo ID, ya que al carga el codHoml, lo puede cargar diferente. Por ejemplo en vez de cargar 5E2, carga 500.

In [21]:
%%time
ids_reps = data_acum_ant['ID'].value_counts()[data_acum_ant['ID'].value_counts() > 1].index.tolist()
for i in ids_reps:
    nombres_sku = data_acum_ant.loc[data_acum_ant['ID'] == i, 'Producto'].unique().tolist()
    mostradores = data_acum_ant.loc[data_acum_ant['ID'] == i, 'Mostrador'].unique().tolist()
    data_acum_ant.loc[data_acum_ant['ID'] == i, 'Producto'] = nombres_sku[-1]
    data_acum_ant.loc[data_acum_ant['ID'] == i, 'Mostrador'] = mostradores[0]

Wall time: 1min 38s


In [22]:
data_acum_ant = data_acum_ant.pivot_table(index=['codHoml', 'Mostrador', 'Código producto', 'Producto'], values=['Cantidad Saldo', 'so'], aggfunc='sum').reset_index()

In [23]:
data_acum_ant.shape

(82212, 6)

In [24]:
data_acum_ant['ID'] = data_acum_ant['codHoml'] + data_acum_ant['Código producto']

In [25]:
data['codHoml'] = data['codHoml'].astype(str)
data['Código producto'] = data['Código producto'].astype(str)
data['ID'] = data['codHoml'] + data['Código producto']

In [26]:
data_desac = pd.merge(data, data_acum_ant, on='ID', how='left')

In [27]:
data_desac['so_x'].sum()

45416.92

In [28]:
data_desac['so_y'].sum()

34736.510485925995

In [29]:
data_desac['so_y'].fillna(0, inplace=True)

In [30]:
data_desac['so'] = data_desac['so_x'] - data_desac['so_y']

In [31]:
data_desac['so'].apply(lambda x: 0 if x < 0 else x).sum()

12936.971250061

In [32]:
data_desac.columns

Index(['codHoml_x', 'Mostrador_x', 'Código producto_x', 'Producto_x',
       'Cantidad Saldo_x', 'so_x', 'ID', 'codHoml_y', 'Mostrador_y',
       'Código producto_y', 'Producto_y', 'Cantidad Saldo_y', 'so_y', 'so'],
      dtype='object')

In [33]:
data_desac.isnull().sum()

codHoml_x               0
Mostrador_x             0
Código producto_x       0
Producto_x              0
Cantidad Saldo_x        0
so_x                    0
ID                      0
codHoml_y            1242
Mostrador_y          1242
Código producto_y    1242
Producto_y           1242
Cantidad Saldo_y     1242
so_y                    0
so                      0
dtype: int64

In [34]:
data_desac_f = data_desac[['codHoml_x', 'Mostrador_x', 'Código producto_x', 'Producto_x', 'Cantidad Saldo_x', 'so']].copy()

In [35]:
rename_f = {'codHoml_x':'codHoml', 'Mostrador_x':'Mostrador', 'Código producto_x':'Código producto', 'Producto_x':'Producto', 'Cantidad Saldo_x':'Cantidad Saldo'}
data_desac_f.rename(rename_f, axis=1, inplace=True)

In [36]:
%%notify -m "¡Listo, mae!"
data_desac_f.isnull().sum()

codHoml            0
Mostrador          0
Código producto    0
Producto           0
Cantidad Saldo     0
so                 0
dtype: int64

<IPython.core.display.Javascript object>

# Exportación de datos

In [44]:
data_desac_f[['codHoml', 'Mostrador', 'Código producto', 'Producto', 'Cantidad Saldo', 'so']].to_excel('../../../../../../../02Paises/Colombia/Copservir/Layout/Copservir_' + str(anio) + '_' + str(semana).zfill(2) + '.xlsx', index=False, sheet_name=str(anio) + '_' + str(semana).zfill(2))

In [37]:
data_desac_f[['Cantidad Saldo', 'so']].sum()

Cantidad Saldo   229061.79
so                10680.41
dtype: float64